# Convolutional Neural Networks
In this notebook we will implement a Convolutional Neural Network. We will be using an architecture from the paper *"ImageNet Classification with Deep Convolutional Neural Networks"*, also known as AlexNet. Such architecture has been considered from many as the one that started the Deep Learning reinassance. We will be using CIFAR10 instead of ImageNet as dataset to reduce computational burden and training time on a single GPU.
Let's start with importing the relevant datasets.

In [15]:
# https://aparico.github.io/

import torch

class PCANorm(nn.Module):
    def __init__(self):
        self.a = 10
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # Flatten the tensor so that you get [[p1, p2, p3], [p1, p2, p3]...]
        # Compute the covariance matrix
        # Compute the SVD to get the eigenvalues
        return x.flatten()


In [6]:
import torch
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader
import torchvision.transforms as T
from matplotlib import pyplot as plt
from torch import nn

# Setup CUDA device
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f'Compute Device: {device}')

# Get the number of cpus so that we can maximize parallel loading
import multiprocessing
num_workers = multiprocessing.cpu_count()


# Load the dataset and return its dataloader
def load_data(root: str, batch_size: int) -> (DataLoader, DataLoader):
    train_data = CIFAR10(root = root, train = True)
    train_data_loader = DataLoader(train_data, batch_size, num_workers, transforms=T.ToTensor)
    test_data = CIFAR10(root = root, train = False)
    test_data_loader = DataLoader(test_data, batch_size, num_workers, transforms=T.ToTensor)
    return (train_data_loader, test_data_loader)

# Data Aumentation is done in 2 ways in the paper:
# 1. By selecting random patches of 224x224
# 2. By applying PCA to something that I still don't know
augmenter = nn.Sequential(
    T.RandomCrop(size = (224, 224))
)

# Define training/test datasets/loaders
batch_size = 8
train_data, test_data = load_data('./resources', batch_size)

# Show an image
img, lbl = train_data.__getitem__(0)
plt.imshow(img)

Compute Device: cuda


TypeError: __init__() got an unexpected keyword argument 'transforms'

In [19]:
import torch.nn 

class PCANorm(nn.Module):

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        raise NotImplementedError
